In [1]:
import os
HOME = os.getcwd()
print(HOME)

/Users/karan/Developer/ChaiBachai


In [2]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.4 torch-2.2.2 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 24.0 GB RAM, 372.6/926.4 GB disk)


In [3]:
from ultralytics import YOLO
import os
import time
import glob
from IPython.display import Image as IPyImage, display
import cv2
import numpy as np

In [4]:
# prompt: give command to fetch a file from my google drive

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [12]:
# trainedModel=f"{HOME}/drive/MyDrive/Tea-boiling-over/best-pot-detection-yolo11s-6.pt"
# testFolder = f"{HOME}/drive/MyDrive/Tea-boiling-over/test-images"
HOME=os.getcwd()
print(HOME)
testImages=f"{HOME}/test-images"
trainedModel=f"{HOME}/best-pot-detection-yolo11s-6.pt"
print(trainedModel)

/Users/karan/Developer/ChaiBachai
/Users/karan/Developer/ChaiBachai/best-pot-detection-yolo11s-6.pt


In [19]:
import torch
import torchvision
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    print("MPS is available")
    mps_device = torch.device("mps")
    model=YOLO(trainedModel)
    model.to(mps_device)

MPS is available


In [28]:
# prompt: how to extract the bounding box dimensions from the images
# !yolo task=detect mode=predict model={trainedModel} conf=0.5 source={testFolder} save=True
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
model=YOLO(trainedModel)

results = model(source=testImages, save=True, device="mps")

for r in results:
    boxes = r.boxes  # Boxes object for bbox outputs
    for box in boxes:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        width = b[2] - b[0]
        height = b[3] - b[1]

        print(f"Bounding box dimensions: Width = {width:.2f}, Height = {height:.2f}")
        # You can access other information from box object like confidence score, class id etc.
        print(box.conf) # confidence score
        print(box.cls)  # class id

NotImplementedError: The operator 'torchvision::nms' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
# prompt: wrap this line to show time a performance
start_time = time.time()
!yolo task=detect mode=predict model={trainedModel} conf=0.5 source={testFolder} save=True
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

In [42]:
# @title Default title text
# prompt: You're absolutely correct that some pots have small notches or spouts, making the rim deviate from a perfect circle. Relying solely on Hough Circle Transform in such cases can lead to inaccuracies. Here's how you can adapt your approach to account for these irregularities:
# 1. Augment Hough Circle Detection with Robust Methods
# While Hough Circle Transform is useful for approximate detection, you can enhance it to handle non-circular rims:
#     Segment the Rim Region Only:
#         Focus on detecting the rim by masking the pot's top portion after detecting the bounding box with YOLOv11s.
#         Use the upper 20–30% of the bounding box as the region for rim detection.
# SAmple code mask = np.zeros_like(edges)
# mask[y_min:int(y_min + 0.3 * box_height), x_min:x_max] = 255
# rim_edges = cv2.bitwise_and(edges, edges, mask=mask)
def detect_rim_with_hough_and_contours(image,boxes):
    detections = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box

        # Crop the region of interest (ROI) around the pot
        ROI = image[y_min:y_max, x_min:x_max]
        # Convert ROI to grayscale and apply Gaussian blur
        gray = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Edge detection using Canny
        edges = cv2.Canny(blurred, 50, 150)

        # Mask the top portion of the pot (top 30% of ROI)
        mask = np.zeros_like(edges)
        box_height = y_max - y_min
        top_mask_height = int(0.3 * box_height)
        mask[:top_mask_height, :] = 255
        rim_edges = cv2.bitwise_and(edges, edges, mask=mask)

        # Apply Hough Circle Transform
        circles = cv2.HoughCircles(
            rim_edges,
            cv2.HOUGH_GRADIENT,dp=1.2,minDist=20,param1=50,param2=30,
            minRadius=10,  # Adjust based on expected pot sizes
            maxRadius=100)

        # Initialize variables
        rim_line = None
        output_image = ROI.copy()

        if circles is not None:
            # Convert circle coordinates to integers
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                x, y, r = circle
                # Draw the detected circle
                cv2.circle(output_image, (x, y), r, (0, 255, 0), 2)
                rim_line = int(y - r)  # Top edge of the circle
                break  # Use the first detected circle

        # If no circle is detected, fall back to contour-based detection
        if rim_line is None:
            # Find contours in the edge-detected image
            contours, _ = cv2.findContours(rim_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                # Find the largest contour
                largest_contour = max(contours, key=cv2.contourArea)
                # Fit a bounding rectangle to the contour and use the top edge
                x, y, w, h = cv2.boundingRect(largest_contour)
                rim_line = y  # Top of the bounding box
                # Draw the contour and bounding rectangle
                cv2.drawContours(output_image, [largest_contour], -1, (255, 0, 0), 2)
                cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Draw the detected rim line on the output image
        if rim_line is not None:
            cv2.line(output_image, (0, rim_line), (output_image.shape[1], rim_line), (0, 0, 255), 2)

        detections.append(rim_line, output_image)
    return detections


In [ ]:


latest_folder = max(glob.glob(f'{testImages}'), key=os.path.getmtime)
for imgPath in glob.glob(f'{latest_folder}/*.jpg')[:]:
    # Load an image (replace 'pot_image.jpg' with your test image)
    image = cv2.imread(imgPath)
    # Example bounding box coordinates from YOLOv11s model
    model=YOLO(trainedModel)
    results = model(image,save=True, imgsz=256, conf=0.5)
    # result = model(image,save=True, imgsz=256, conf=0.5, device="mps")

    pot_bboxes=[]
    for result in results:
        pot_bboxees=result.boxes.xyxy

    # Detect the rim
    detections = detect_rim_with_hough_and_contours(image, pot_bboxes)
    print("det",len(detections))
    for detection in detections:
        print("\n\n\ndetections")
        print(detection)
        rim_line, output_image = detection
        # Display the result
        cv2.imshow("Detected Rim", output_image)

    cv2.waitKey(0)

In [ ]:


latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:]:
    display(IPyImage(filename=img, width=600))
    print("\n")

In [ ]:
import cv2
import numpy as np

def detect_rim_with_hough_and_contours(image, pot_bbox):
    """
    Detects the rim of a pot using Hough Circle Transform augmented with contour analysis.

    Args:
        image: Input image (BGR).
        pot_bbox: Tuple containing bounding box coordinates of the pot (x_min, y_min, x_max, y_max).

    Returns:
        rim_line: Y-coordinate of the detected rim.
        output_image: Image with rim detection drawn.
    """
    # Extract bounding box coordinates
    x_min, y_min, x_max, y_max = pot_bbox

    # Crop the region of interest (ROI) around the pot
    ROI = image[y_min:y_max, x_min:x_max]

    # Convert ROI to grayscale and apply Gaussian blur
    gray = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Mask the top portion of the pot (top 30% of ROI)
    mask = np.zeros_like(edges)
    box_height = y_max - y_min
    top_mask_height = int(0.3 * box_height)
    mask[:top_mask_height, :] = 255
    rim_edges = cv2.bitwise_and(edges, edges, mask=mask)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(
        rim_edges,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=20,
        param1=50,
        param2=30,
        minRadius=10,  # Adjust based on expected pot sizes
        maxRadius=100
    )

    # Initialize variables
    rim_line = None
    output_image = ROI.copy()

    if circles is not None:
        # Convert circle coordinates to integers
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            x, y, r = circle
            # Draw the detected circle
            cv2.circle(output_image, (x, y), r, (0, 255, 0), 2)
            rim_line = int(y - r)  # Top edge of the circle
            break  # Use the first detected circle

    # If no circle is detected, fall back to contour-based detection
    if rim_line is None:
        # Find contours in the edge-detected image
        contours, _ = cv2.findContours(rim_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            # Find the largest contour
            largest_contour = max(contours, key=cv2.contourArea)
            # Fit a bounding rectangle to the contour and use the top edge
            x, y, w, h = cv2.boundingRect(largest_contour)
            rim_line = y  # Top of the bounding box
            # Draw the contour and bounding rectangle
            cv2.drawContours(output_image, [largest_contour], -1, (255, 0, 0), 2)
            cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Draw the detected rim line on the output image
    if rim_line is not None:
        cv2.line(output_image, (0, rim_line), (output_image.shape[1], rim_line), (0, 0, 255), 2)

    return rim_line, output_image


# Example usage
if __name__ == "__main__":
    # Load an image (replace 'pot_image.jpg' with your test image)
    image = cv2.imread("pot_image.jpg")

    # Example bounding box coordinates from YOLOv11s model
    pot_bbox = (50, 100, 300, 400)  # Replace with actual YOLO output

    # Detect the rim
    rim_line, output_image = detect_rim_with_hough_and_contours(image, pot_bbox)

    # Display the result
    cv2.imshow("Detected Rim", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
